In [ ]:
import numpy as np
import pandas as pd
import gc

In [ ]:
order_products_prior = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/order_products__prior.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/orders.csv')

In [ ]:
# объединиться с файлом заказов, чтобы получить user_id
prior_df = pd.merge(order_products_prior, orders, how="inner", on="order_id")
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [ ]:
del order_products_prior

In [ ]:
# получить продукты и статус повторных заказов последней покупки каждого пользователя
prior_grouped_df = prior_df.groupby("user_id")["order_number"].aggregate("max").reset_index()
prior_df_latest = pd.merge(prior_df, prior_grouped_df, how="inner", on=["user_id", "order_number"])
prior_df_latest = prior_df_latest[["user_id", "product_id", "reordered"]]
prior_df_latest.columns = ["user_id", "product_id", "reordered_latest"]
prior_df_latest.head()

,user_id,product_id,reordered_latest
0,59897,9755,1
1,59897,31487,0
2,59897,37510,1
3,59897,14576,1
4,59897,22105,0


In [ ]:
# получить количество каждого продукта и количество повторных заказов со стороны клиента
prior_df_agg = prior_df.groupby(["user_id","product_id"])["reordered"].aggregate(["count", "sum"]).reset_index()
prior_df_agg.columns = ["user_id", "product_id", "reordered_count", "reordered_sum"]
prior_df_agg.head()

,user_id,product_id,reordered_count,reordered_sum
0,1,196,10,9
1,1,10258,9,8
2,1,10326,1,0
3,1,12427,10,9
4,1,13032,3,2


In [ ]:
prior_df_merge = pd.merge(prior_df_agg, prior_df_latest, how="left", on=["user_id","product_id"])
prior_df_merge.head()

,user_id,product_id,reordered_count,reordered_sum,reordered_latest
0,1,196,10,9,1.0
1,1,10258,9,8,1.0
2,1,10326,1,0,NaN
3,1,12427,10,9,1.0
4,1,13032,3,2,1.0


In [ ]:
prior_df_merge

,user_id,product_id,reordered_count,reordered_sum,reordered_latest
0,1,196,10,9,1.0
1,1,10258,9,8,1.0
2,1,10326,1,0,NaN
3,1,12427,10,9,1.0
4,1,13032,3,2,1.0
...,...,...,...,...,...
13307948,206209,43961,3,2,NaN
13307949,206209,44325,1,0,NaN
13307950,206209,48370,1,0,NaN
13307951,206209,48697,1,0,NaN


In [ ]:
orders.drop(["order_number"],axis=1,inplace=True)

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/order_products__train.csv", usecols=["order_id"])
train_df.head()

,order_id
0,1
1,1
2,1
3,1
4,1


In [ ]:
train_df = train_df.groupby("order_id").aggregate("count").reset_index()
train_df.head()

,order_id
0,1
1,36
2,38
3,96
4,98


In [ ]:
train_df = pd.merge(train_df, orders, how="left", on="order_id")
train_df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order
0,1,112108,train,4,10,9.0
1,36,79431,train,6,18,30.0
2,38,42756,train,6,16,24.0
3,96,17227,train,6,20,30.0
4,98,56463,train,3,8,14.0


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/sample_submission.csv", usecols=["order_id"])

test_df = pd.merge(test_df, orders, how="inner", on="order_id")
test_df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order
0,17,36855,test,6,15,1.0
1,34,35220,test,3,11,8.0
2,137,187107,test,2,19,30.0
3,182,115892,test,0,11,8.0
4,257,35581,test,6,23,5.0


In [ ]:
train_df

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order
0,1,112108,train,4,10,9.0
1,36,79431,train,6,18,30.0
2,38,42756,train,6,16,24.0
3,96,17227,train,6,20,30.0
4,98,56463,train,3,8,14.0
...,...,...,...,...,...,...
131204,3421049,189544,train,1,9,3.0
131205,3421056,83898,train,2,20,11.0
131206,3421058,136952,train,3,18,15.0
131207,3421063,169679,train,0,10,4.0


Подготовка тестовых

In [ ]:
print(train_df.shape, test_df.shape)

(131209, 6) (75000, 6)


Объединение данные обучения и тестовых с Prior_df, чтобы получить продукты, приобретенные клиентом ранее

In [ ]:
train_df = pd.merge(train_df, prior_df_merge, how="inner", on="user_id")
train_df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest
0,1,112108,train,4,10,9.0,2067,1,0,NaN
1,1,112108,train,4,10,9.0,5707,2,1,1.0
2,1,112108,train,4,10,9.0,11109,2,1,NaN
3,1,112108,train,4,10,9.0,14947,3,2,1.0
4,1,112108,train,4,10,9.0,22035,2,1,NaN


In [ ]:
test_df = pd.merge(test_df, prior_df_merge, how="inner", on="user_id")
test_df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest
0,17,36855,test,6,15,1.0,1283,1,0,NaN
1,17,36855,test,6,15,1.0,6291,1,0,NaN
2,17,36855,test,6,15,1.0,7035,1,0,NaN
3,17,36855,test,6,15,1.0,11494,1,0,NaN
4,17,36855,test,6,15,1.0,13107,3,2,1.0


In [ ]:
del prior_df, prior_grouped_df, prior_df_latest

In [ ]:
products_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/products.csv', usecols=["product_id", "aisle_id", "department_id"])
products_df.head()

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13


Добавлю дополнительные функции к обучающим данным из фрейма данных «Продукты».

In [ ]:
train_df = pd.merge(train_df, products_df, how="inner", on="product_id")
train_df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
0,1,112108,train,4,10,9.0,2067,1,0,NaN,3,19
1,96,17227,train,6,20,30.0,2067,1,0,NaN,3,19
2,3243,206024,train,3,15,2.0,2067,1,0,NaN,3,19
3,12950,11456,train,1,14,13.0,2067,1,0,0.0,3,19
4,17683,177724,train,5,21,0.0,2067,1,0,NaN,3,19


In [ ]:
test_df = pd.merge(test_df, products_df, how="inner", on="product_id")
test_df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
0,17,36855,test,6,15,1.0,1283,1,0,NaN,100,21
1,657743,16994,test,1,9,11.0,1283,1,0,NaN,100,21
2,834223,92628,test,6,14,11.0,1283,1,0,NaN,100,21
3,1446886,75870,test,0,10,9.0,1283,1,0,NaN,100,21
4,1518981,118458,test,5,15,6.0,1283,3,2,NaN,100,21


Теперь у нас есть все продукты, которые ранее были приобретены клиентом, а также некоторые характеристики/особенности. Таким образом, мы можем использовать набор данных обучения для заполнения целевой переменной, т. е. того, был ли продукт переупорядочен в следующем заказе.

In [ ]:
train_y_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/order_products__train.csv", usecols=["order_id", "product_id", "reordered"])
train_y_df = pd.merge(train_y_df, orders, how="inner", on="order_id")
train_y_df = train_y_df[["user_id", "product_id", "reordered"]]
train_df = pd.merge(train_df, train_y_df, how="left", on=["user_id", "product_id"])
train_df["reordered"].fillna(0, inplace=True)
print(train_df.shape)

(8474661, 13)


In [ ]:
del train_y_df, products_df, orders, prior_df_merge, prior_df_agg

In [ ]:
X=train_df.drop('reordered',axis=1)

In [ ]:
X = X.drop('eval_set',axis=1)

In [ ]:
X.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
0,1,112108,4,10,9.0,2067,1,0,NaN,3,19
1,96,17227,6,20,30.0,2067,1,0,NaN,3,19
2,3243,206024,3,15,2.0,2067,1,0,NaN,3,19
3,12950,11456,1,14,13.0,2067,1,0,0.0,3,19
4,17683,177724,5,21,0.0,2067,1,0,NaN,3,19


In [ ]:
Y = train_df['reordered']

In [ ]:
Y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: reordered, dtype: float64

In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/train_df.csv')

In [ ]:
del train_df

In [ ]:
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/test_df.csv')